<a href="https://colab.research.google.com/github/SayiaFelix/natural-language-processing/blob/main/Natural_Language_Processing_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = ':https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F39657%2F61725%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240802%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240802T084738Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6db3ba8f7d3864ce839c6fd521ec1347388b098a44444982a93a1491bf8b8d2e452d206c3b5c811830dd092f4a04f45e824e6fa35233f69922fee9a8c3358db295deb38d2f43b1147fdc3f34faf40cc0ad11e7abbf82c377d19113a66eb938d7ccf4e18656e4a17459e8ff58c81baed98e6f0e977b3efac95bdd61cd486c768f152523212bd9b508623f160a0a0941f52dd29540d36a9d58c44fdf750496fae5ea5155b3131061990a83342641adb38826e34db2be8040e3e6d9084f69cf634890aed20076f68740df0c3150bfba2612d76d6d65abc1781af959f697b79df2e7cee384ceb5ef5879305d9db1d1af6d1917193cb93275e43c9236472a5a4be627'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 167607 bytes downloaded
Downloaded and uncompressed: 
Data source import complete.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['amazon_alexa.tsv']


In [4]:
#Using rstrip() plus a list comprehension to get a list of all the lines of text review

In [5]:
df_review = [line.rstrip() for line in open('../input/amazon_alexa.tsv')]
print (len(df_review))

3151


In [ ]:
import pandas
df_review = pandas.read_csv('../input/amazon_alexa.tsv', sep='\t')
df_review.head()

**Basic Exploratory Data Analysis**
Let's check out some of the stats with some plots and the built-in methods in pandas!

In [ ]:
df_review.describe()

In [ ]:
df_review.groupby('rating').describe()

Making a new column to detect how long the text messages are:

In [ ]:
df_review['length'] = df_review['verified_reviews'].apply(len)
df_review.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
df_review['length'].plot(bins=50, kind='hist')

Finding out the Maximum length review

In [ ]:
df_review.length.describe()

Wow! 2851 characters long review, let's use masking to find this message:



In [ ]:
df_review[df_review['length'] == 2851]['verified_reviews'].iloc[0]

**Looks like we have some sort of Essay writing review! Now let's focus back on the idea of trying to see if review length is a distinguishing feature between positive and negative review:**

In [ ]:
df_review.hist(column='length', by='feedback', bins=50,figsize=(10,4))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Importing the dataset
dataset = pd.read_csv('../input/amazon_alexa.tsv', delimiter = '\t', quoting = 3)


In [ ]:
# Cleaning the texts
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus=[]
for i in range(0,3150):
    review = re.sub('[^a-zA-Z]', ' ', dataset['verified_reviews'][i] )
    review=review.lower()
    review=review.split()
    ps=PorterStemmer()
    review=[ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review=' '.join(review)
    corpus.append(review)

In [ ]:
# creating the Bag of words Model
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=1500)
X=cv.fit_transform(corpus).toarray()
y=dataset.iloc[:,4].values

In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)


In [ ]:
# Fitting Random Forest classifier with 100 trees to the Training set
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 100, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)


In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [ ]:
cm

Above shown matrix is known as confusion matrix and below is my result with accuracy of 94.28% and F1 Score of 0.9696.The Random Classifier Algo with 100 trees works efficiently to train the machine in predicting positive and negative reviews.
<img src="https://image.ibb.co/hwRArK/claass.png" alt="claass" border="0">

<img src="https://image.ibb.co/jK2Byz/Precisionrecall.png" alt="Precisionrecall" border="0">

<img src="https://image.ibb.co/bU3ZJz/confusion_matrix_1.png" alt="confusion_matrix_1" border="0">

**Can You more efficiently train your machine model and Beat my F1 score of 0.9696 using any other optimized classifier algo**